<a href="https://colab.research.google.com/github/crystalloide/Notebooks-SQL-serie-2/blob/main/R%C3%A9ponses/Cours_SQL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL 2/3
> Cours IMT Mines 02-2022

- categories: [Cours IMT Mines 02-2022, Bases de données, SGBR-R, SQL, TP]

# Aller plus loin en SQL

Dans ce classeur, nous poursuivrons l'étude de l'exemple de la base de données de livres de SF et nous verrons des requêtes de sélection avancées ainsi que la manière de fusionner plusieurs tables (Jointures).

Commencez par valider les cellules suivantes afin d'activer la fonctionnalité SQL de Jupyter et de charger la base du TP précédent.

In [ ]:
%load_ext sql

In [ ]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
%sql sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db

'Connected: @/content/gdrive/MyDrive/Partage/livres_AF.db'

In [ ]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


name
Langues
Auteurs
Livres
Themes
RelationsLivreTheme


## Aller plus loin avec SELECT

### Sélection simple

Supposons que l'on veuille lister seulement les noms et prénoms des auteurs nés avant 1900, on utilisera une clause **WHERE** *condition* :

In [ ]:
%sql SELECT Nom, Prenom from Auteurs WHERE AnneeNaissance < 1900;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


Nom,Prenom
Huxley,Aldous
Verne,Jules


Pour obtebir les auteurs prénommés *Jules* :

In [ ]:
%sql SELECT Nom, Prenom from Auteurs WHERE Prenom= 'Jules';

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


Nom,Prenom
Verne,Jules


Vous voyez que les textes sont délimités par des quotes **'...'**.

***Attention*** : essayez de remplacer **Jules** par **jules** et vous constaterez que la recherche est sensible aux majuscules.

Supposons maintenant que l'on recherche les auteurs dont le prénom commence par **P** : Nous utiliserons une clause *LIKE* de cette manière :

In [ ]:
%sql SELECT Nom, Prenom from Auteurs WHERE Prenom LIKE 'P%';

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


Nom,Prenom
K. Dick,Philip
Boulle,Pierre


### Sélection multiple
Il est possible de croiser plusieurs critères à l'aide d'opérateurs booleens : **AND** et **OR**. Voici la liste des auteurs français nés après 1900 :

In [ ]:
%sql SELECT Nom, Prenom from Auteurs WHERE IdLangue = 2 AND AnneeNaissance > 1900;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


Nom,Prenom
Barjavel,René
Boulle,Pierre


### Compter le nombre de réponses d'une requête SELECT

Combien y a t-il d'auteurs nés entre 1900 et 1915 ? Vous verrez à l'occation l'utilisation de l'opérateur *BETWEEN* pour tester l'appartenance à un intervalle.

In [ ]:
nbr = %sql SELECT COUNT(*) as Total from Auteurs WHERE AnneeNaissance BETWEEN 1900 AND 1915;

print(nbr)
print("récupérer juste le nombre : ", nbr[0][0])

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.
+-------+
| Total |
+-------+
|   4   |
+-------+
récupérer juste le nombre :  4


### Trier les réponses

Nous allons lister tous les auteurs par ordre croissant d'année de naissance

In [ ]:
%sql SELECT * from Auteurs ORDER BY AnneeNaissance;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


IdAuteur,Nom,Prenom,AnneeNaissance,IdLangue
10,Verne,Jules,1828,2
4,Huxley,Aldous,1894,1
1,Orwell,George,1903,1
7,Barjavel,René,1911,2
8,Boulle,Pierre,1912,2
9,Van Vogt,Alfred Elton,1912,1
2,Herbert,Frank,1920,1
3,Asimov,Isaac,1920,1
5,Bradbury,Ray,1920,1
6,K. Dick,Philip,1928,1


et par ordre décroissant, on ajoute **DESC** à la fin de la requête

In [ ]:
%sql SELECT * from Auteurs ORDER BY AnneeNaissance DESC;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


IdAuteur,Nom,Prenom,AnneeNaissance,IdLangue
6,K. Dick,Philip,1928,1
2,Herbert,Frank,1920,1
3,Asimov,Isaac,1920,1
5,Bradbury,Ray,1920,1
8,Boulle,Pierre,1912,2
9,Van Vogt,Alfred Elton,1912,1
7,Barjavel,René,1911,2
1,Orwell,George,1903,1
4,Huxley,Aldous,1894,1
10,Verne,Jules,1828,2


### Éviter les occurrences multiples

interrogeons la table **Livres** sur les années de publication, rangées par ordre croissant :

In [ ]:
%sql SELECT AnneePubli from Livres ORDER BY AnneePubli;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


AnneePubli
1865
1931
1943
1945
1949
1950
1950
1951
1953
1955


On constate la présence de quelques doublons

Pour éviter les redondances dans les résultats, on peut rajouter le mot-clé *DISTINCT* juste après le *SELECT* :

In [ ]:
%sql SELECT DISTINCT AnneePubli from Livres ORDER BY AnneePubli;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


AnneePubli
1865
1931
1943
1945
1949
1950
1951
1953
1955
1962


### A vous de jouer

1. Donner la liste de tous les titres des livres écrits entre 1920 et 1950.
1. Combien y en a t-il ?

In [ ]:
# Utilisez cette cellule en brouillon pour vos requetes 

In [ ]:
# Utiliser la syntaxe courte 
reponse_1 = %sql SELECT Titre from Livres WHERE AnneePubli >=1920 AND AnneePubli<=1950
reponse_2 = %sql SELECT COUNT(*) as Total from Livres WHERE AnneePubli >=1920 AND AnneePubli<=1950


 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.
 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


In [ ]:
print(reponse_1)
print(reponse_2)

+------------------------+
|         Titre          |
+------------------------+
|          1984          |
| Le meilleur des mondes |
| Chroniques martiennes  |
|       Les Robots       |
|         Ravage         |
|     Le monde des A     |
+------------------------+
+-------+
| Total |
+-------+
|   6   |
+-------+


In [ ]:
# Vérification des réponses

assert reponse_1[3][0] == 'Les Robots'
assert reponse_2[0][0] == 6


## Requêtes portant sur plusieurs tables

Jusqu'à présent, nos requêtes ne portaient que sur une seule table. Néanmoins notre liste de livres comporte des donées en provenance de plusieurs tables simultanément. Nous allons voir comment effectuer des requêtes pour croiser des données en provenance de plusieurs tables.

In [ ]:
%sql SELECT * FROM Langues;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


IdLangue,Langue
1,Anglais
2,Français


In [ ]:
%sql SELECT * FROM Auteurs;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


IdAuteur,Nom,Prenom,AnneeNaissance,IdLangue
1,Orwell,George,1903,1
2,Herbert,Frank,1920,1
3,Asimov,Isaac,1920,1
4,Huxley,Aldous,1894,1
5,Bradbury,Ray,1920,1
6,K. Dick,Philip,1928,1
7,Barjavel,René,1911,2
8,Boulle,Pierre,1912,2
9,Van Vogt,Alfred Elton,1912,1
10,Verne,Jules,1828,2


In [ ]:
%sql SELECT * FROM Langues, Auteurs;

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


IdLangue,Langue,IdAuteur,Nom,Prenom,AnneeNaissance,IdLangue_1
1,Anglais,1,Orwell,George,1903,1
1,Anglais,2,Herbert,Frank,1920,1
1,Anglais,3,Asimov,Isaac,1920,1
1,Anglais,4,Huxley,Aldous,1894,1
1,Anglais,5,Bradbury,Ray,1920,1
1,Anglais,6,K. Dick,Philip,1928,1
1,Anglais,7,Barjavel,René,1911,2
1,Anglais,8,Boulle,Pierre,1912,2
1,Anglais,9,Van Vogt,Alfred Elton,1912,1
1,Anglais,10,Verne,Jules,1828,2


Comme on peut le constater cette requête est peu pertinente car elle affiche toutes les données de chacune des tables sans effectuer de correspondances. La **clé de jointure** apparaît pourtant ici clairement : il s'agit de **id_langue** qui doit permettre de recouper les informations entre les deux tables : il est en effet inutile d'afficher les données pour lesquelles les langues ne correspondent pas entre les deux tables.

### Jointure

La **jonture** consiste à croiser les données de plusieurs tables pour les présenter sous forme d'un seul tableau. On va utiliser ce mécanisme pour afficher clairement la langue de l'auteur plutôt qu'un numéro qui n'est pas forcément très parlant. Nous utiliserons pour cela l'opérateur **JOIN ... ON** :

In [ ]:
%sql SELECT Nom, Prenom, Langue, AnneeNaissance  FROM Auteurs JOIN Langues ON Auteurs.IdLangue = Langues.IdLangue

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


Nom,Prenom,Langue,AnneeNaissance
Orwell,George,Anglais,1903
Herbert,Frank,Anglais,1920
Asimov,Isaac,Anglais,1920
Huxley,Aldous,Anglais,1894
Bradbury,Ray,Anglais,1920
K. Dick,Philip,Anglais,1928
Barjavel,René,Français,1911
Boulle,Pierre,Français,1912
Van Vogt,Alfred Elton,Anglais,1912
Verne,Jules,Français,1828


Les champs sur lesquels faire la jointure **ayant les mêmes noms dans les 2 tables**, cette requête peut aussi être écrite plus simplement en utilisant le mot-clé **USING** ainsi : 

In [ ]:
%sql SELECT Nom, Prenom, Langue, AnneeNaissance  FROM Auteurs JOIN Langues USING (IdLangue)

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


Nom,Prenom,Langue,AnneeNaissance
Orwell,George,Anglais,1903
Herbert,Frank,Anglais,1920
Asimov,Isaac,Anglais,1920
Huxley,Aldous,Anglais,1894
Bradbury,Ray,Anglais,1920
K. Dick,Philip,Anglais,1928
Barjavel,René,Français,1911
Boulle,Pierre,Français,1912
Van Vogt,Alfred Elton,Anglais,1912
Verne,Jules,Français,1828


### A vous de jouer
En croisant la table **Livres** avec la table **Auteurs**

1. récupérer dans la variable `reponse_1` une liste dont les attributs sont **Titre**, **PrenomAuteur**, **NomAuteur** et **AnneePubli**, triée du plus récent au plus ancien.
1. récupérer dans la variable `reponse_2` une liste dont les attributs sont **Titre**, **PrenomAuteur**, **NomAuteur** et **AnneePubli** écrits en français.

In [ ]:
# Utiliser la syntaxe courte 
reponse_1 = %sql select Titre, Prenom, Nom, AnneePubli from Livres join Auteurs using (IdAuteur) order by AnneePubli DESC
reponse_2 = %sql select Titre, Prenom, Nom, AnneePubli from Livres join Auteurs using (IdAuteur) where Auteurs.IdLangue == 2

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.
 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


In [ ]:
print(reponse_1)
print(reponse_2)

+---------------------------+--------------+----------+------------+
|           Titre           |    Prenom    |   Nom    | AnneePubli |
+---------------------------+--------------+----------+------------+
|            Ubik           |    Philip    | K. Dick  |    1969    |
|     La nuit des temps     |     René     | Barjavel |    1968    |
|        Blade Runner       |    Philip    | K. Dick  |    1968    |
|            Dune           |    Frank     | Herbert  |    1965    |
|   La Planète des singes   |    Pierre    |  Boulle  |    1963    |
| Le Maître du Haut Château |    Philip    | K. Dick  |    1962    |
|    La Fin de l’éternité   |    Isaac     |  Asimov  |    1955    |
|       Fahrenheit 451      |     Ray      | Bradbury |    1953    |
|         Fondation         |    Isaac     |  Asimov  |    1951    |
|   Chroniques martiennes   |     Ray      | Bradbury |    1950    |
|         Les Robots        |    Isaac     |  Asimov  |    1950    |
|            1984           |    G

In [ ]:
assert reponse_1[0] == ('Ubik', 'Philip', 'K. Dick', 1969)
assert reponse_2[0] == ('La nuit des temps', 'René', 'Barjavel', 1968)

### Le cas des relations de n à n

Parfois il arrive que les données à collecter se trouvent dans plus que deux tables : c'est le cas des Thèmes pour les livres qui nécessitent l'analyse de 3 tables : **Livres** et **Thèmes** bien sûr, mais aussi la table de relation **RelationsLivreTheme**.

Observez et étudiez la requête suivante : Le principe est d'enchaîner deux jointures **JOIN ... USING** en utilisant ***la table de Relation au milieu***. En effet, la requête se lit de la gauche vers la droite et on ne peut faire de jointure que si on a une clé externe en commun, ce qui n'est par exemple pas le cas entre **Livres** et **Thèmes**.

In [ ]:
%sql SELECT Titre, Intitule FROM Livres JOIN RelationsLivreTheme USING (IdLivre) JOIN Themes USING (IdTheme)

 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


Titre,Intitule
1984,Totalitarisme
1984,science-fiction
1984,anticipation
1984,dystopie
Dune,science-fiction
Dune,anticipation
Fondation,science-fiction
Fondation,économie
Le meilleur des mondes,Totalitarisme
Le meilleur des mondes,science-fiction


### A vous de jouer

Ecrire une requête permettant d'obtenir une liste dont les attributs sont **Titre**, **NomAuteur** et **Langue** triée par ordre croissant de date de naissance de l'auteur.

Vous stockerez le résultat dans une variable `result`

In [ ]:
# Utiliser la syntaxe courte 
reponse = %sql SELECT Titre, Nom, Langue FROM Livres JOIN Auteurs USING (IdAuteur) JOIN Langues USING (IdLangue) ORDER BY Auteurs.AnneeNaissance


 * sqlite:////content/gdrive/MyDrive/Partage/livres_AF.db
Done.


In [ ]:
print (reponse)

+---------------------------+----------+----------+
|           Titre           |   Nom    |  Langue  |
+---------------------------+----------+----------+
|   De la Terre à la Lune   |  Verne   | Français |
|   Le meilleur des mondes  |  Huxley  | Anglais  |
|            1984           |  Orwell  | Anglais  |
|     La nuit des temps     | Barjavel | Français |
|           Ravage          | Barjavel | Français |
|   La Planète des singes   |  Boulle  | Français |
|       Le monde des A      | Van Vogt | Anglais  |
|            Dune           | Herbert  | Anglais  |
|         Fondation         |  Asimov  | Anglais  |
|       Fahrenheit 451      | Bradbury | Anglais  |
|   Chroniques martiennes   | Bradbury | Anglais  |
|         Les Robots        |  Asimov  | Anglais  |
|    La Fin de l’éternité   |  Asimov  | Anglais  |
|            Ubik           | K. Dick  | Anglais  |
|        Blade Runner       | K. Dick  | Anglais  |
| Le Maître du Haut Château | K. Dick  | Anglais  |
+-----------

In [ ]:
assert reponse[0] == ('De la Terre à la Lune', 'Verne', 'Français')
